In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


In [2]:
# Check and set GPU configuration
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)


In [6]:
# Define paths
train_dir = "D:/university/cse400(A,B,C)/Dataset/skin/archive/Dataset1.0/train"
test_dir = "D:/university/cse400(A,B,C)/Dataset/skin/archive/Dataset1.0/test"

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to [0, 1]
    rotation_range=40,         # Randomly rotate images by up to 40 degrees
    width_shift_range=0.2,     # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2,    # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2,           # Apply random shearing transformations
    zoom_range=0.2,            # Randomly zoom into images by up to 20%
    horizontal_flip=True,      # Randomly flip images horizontally
    vertical_flip=False,       # Disable vertical flipping (optional, can be enabled if needed)
    brightness_range=[0.8, 1.2],  # Randomly adjust brightness
    fill_mode='nearest',       # Strategy for filling in newly created pixels (nearest neighbor interpolation)
    validation_split=0.2       # Use 20% of the data for validation
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 8359 images belonging to 10 classes.
Found 2088 images belonging to 10 classes.
Found 3000 images belonging to 10 classes.


In [7]:
# Load the ResNet50 model with pre-trained weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [8]:
# Train the model
history = model.fit(
    train_generator,
    epochs=200,
    validation_data=validation_generator,
    verbose=1
)


Epoch 1/200
262/262 [==============================] - 138s 515ms/step - loss: 2.3196 - accuracy: 0.1058 - val_loss: 2.3128 - val_accuracy: 0.0939
Epoch 2/200
262/262 [==============================] - 148s 566ms/step - loss: 2.3100 - accuracy: 0.1170 - val_loss: 2.2887 - val_accuracy: 0.1355
Epoch 3/200
262/262 [==============================] - 141s 539ms/step - loss: 2.2917 - accuracy: 0.1318 - val_loss: 2.2869 - val_accuracy: 0.1216
Epoch 4/200
262/262 [==============================] - 133s 506ms/step - loss: 2.2794 - accuracy: 0.1297 - val_loss: 2.2723 - val_accuracy: 0.1384
Epoch 5/200
262/262 [==============================] - 142s 541ms/step - loss: 2.2698 - accuracy: 0.1370 - val_loss: 2.2747 - val_accuracy: 0.1628
Epoch 6/200
262/262 [==============================] - 142s 542ms/step - loss: 2.2580 - accuracy: 0.1473 - val_loss: 2.2788 - val_accuracy: 0.1231
Epoch 7/200
262/262 [==============================] - 136s 520ms/step - loss: 2.2456 - accuracy: 0.1579 - val_loss: 2

KeyboardInterrupt: 

In [ ]:
# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model again with fine-tuning
history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print(f"Test accuracy: {test_accuracy:.2f}")


In [ ]:
# Generate predictions
Y_pred = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
y_pred_classes = np.argmax(Y_pred, axis=1)

# True labels
y_true = test_generator.classes

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
class_names = list(test_generator.class_indices.keys())

# Classification Report
class_report = classification_report(y_true, y_pred_classes, target_names=class_names)

print("Classification Report:")
print(class_report)

# Plot Confusion Matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Plot training and validation accuracy
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
